## 0. Import packages

In [14]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
import numpy as np
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [2]:
import pickle
rules = pickle.load(open("pickle_dumps/article_rules.p", "rb"))
df_test = pickle.load(open("pickle_dumps/test_df.p", "rb"))
df_test = df_test.set_index("user_id").to_dict()["item_id"]

In [3]:
print(len(rules))
for i in range(10):
    if i < len(rules):
        print(rules[i])

9112
Conf: 0.96	Supp: 26.00	 {1025, 2954, 132} => {7}
Conf: 0.95	Supp: 20.00	 {562, 20396, 23214} => {7}
Conf: 0.93	Supp: 27.00	 {1025, 3, 132} => {7}
Conf: 0.92	Supp: 34.00	 {1025, 171, 132} => {7}
Conf: 0.92	Supp: 23.00	 {2219, 14364, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 2219} => {7}
Conf: 0.92	Supp: 23.00	 {3, 2219, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 3327} => {7}
Conf: 0.92	Supp: 22.00	 {129, 2219, 1025} => {7}
Conf: 0.91	Supp: 21.00	 {1025, 7, 23047} => {132}


In [4]:
class Recommender():
    def __init__(self, df_train, assos, df_test):
        pass
    
    def recommend(self, user_id, k):
        pass        

In [5]:
class Assos_Recommender(Recommender):
    def __init__(self, assos):
        # use popular items to fill unused space
        # TODO (df_train is used for this)
        
        self.assos = assos
        
#     def get_items(self, user_id):
#         return self.df_test[user_id]
    
    def recommend(self, user_items, k):
        recommendation = set()
        for a in self.assos:
            for l in a.left:
                # break if any element is not in the user session
                if l not in user_items:
                    break
                    
                # all elements are in the user session --> add RHS of rule to the recommendation
                for item in a.right:
                    r = item
                    if r not in user_items and (len(recommendation) < k or k == -1):
                        recommendation.add(r)
                        
            if len(recommendation) >= k and k != -1:
                break
                
        return recommendation

In [24]:
def splitUser(user_items):
    splits = []
    
#     scores = cross_val_score(clf, X, y, cv=5)
    tmp = np.array_split(user_items, 4)
    
    for l in tmp:
        if len(l) > 0:
            splits.append(list(l))
    
    res = []
    for i in splits:
        join = []
        for j in splits:
            if i != j:
                join += j
        res.append((join, i))
                    
                

    return res

print(splitUser(range(10)))

[([3, 4, 5, 6, 7, 8, 9], [0, 1, 2]), ([0, 1, 2, 6, 7, 8, 9], [3, 4, 5]), ([0, 1, 2, 3, 4, 5, 8, 9], [6, 7]), ([0, 1, 2, 3, 4, 5, 6, 7], [8, 9])]


In [28]:
def hitrate(rc, k):
    hits = 0
    counts = 0
    avg_len = 0
    for user in tqdm(df_test):
        user_items = df_test[user]
        split_user = splitUser(user_items)
        for split in split_user:
            r = rc.recommend(split[0], k)
        
            avg_len += len(r)/len(split_user)
            
            for item in split[1]:
                if item in r:
                    hits += (item in r) / len(split_user)
                    break
                
        counts += 1
    
    print("avg len: {:f}".format(avg_len / len(df_test)))
    if k == -1:
        print("HR\t{:.5f}".format((hits / counts)))
    else:
        print("HR@{}\t{:.5f}".format(k, (hits / counts)))
    return hits / counts

In [26]:
rc = Assos_Recommender(rules)
hr = hitrate(rc, -1)

100%|██████████| 11256/11256 [02:35<00:00, 72.43it/s]

avg len: 1.118507
HR	0.01481%


In [27]:
print(hr)

0.00014806917791992418


# ??. Store the association rules as pickle file for further use

In [10]:
# import pickle
# pickle.dump(rules, open("pickle_dumps/article_rules.p", "wb"))